# 11 — Advanced Visualization & Dash Dashboards (EN/PT)

This notebook demonstrates high-performance interactive visualizations with Plotly and Dash, integrates project models, and shows local/cloud deployment.

Este notebook demonstra visualizações interativas de alta performance com Plotly e Dash, integra modelos do projeto e mostra publicação local/na nuvem.

## Contents | Conteúdo
- Setup (Plotly, Dash) | Preparação (Plotly, Dash)
- High-performance charts (WebGL, density, large data) | Gráficos de alta performance (WebGL, densidade, dados grandes)
- Model integration examples | Exemplos de integração com modelos
- Dash app structure | Estrutura de app Dash
- Local run and cloud deploy (Render, Railway, Azure) | Execução local e deploy em nuvem (Render, Railway, Azure)

In [ ]:
# %% Setup / Preparação
!pip -q install plotly dash dash-bootstrap-components pandas numpy
import os, json, numpy as np, pandas as pd
import plotly.express as px
from datetime import datetime

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
print('Versions:', {'plotly': px.__version__})

## High-performance interactive charts | Gráficos interativos de alta performance
We use WebGL traces and data aggregation for responsiveness. | Usamos traçados WebGL e agregação para responsividade.

In [ ]:
# Large scatter with WebGL
N = 150_000
df = pd.DataFrame({
    'x': np.random.randn(N).cumsum(),
    'y': np.random.randn(N).cumsum(),
    'group': np.random.choice(list('ABCDE'), size=N)
})
fig_gl = px.scatter(df, x='x', y='y', color='group', render_mode='webgl',
                   title='WebGL scatter — 150k points')
fig_gl.show()

In [ ]:
# Density heatmap for large data
fig_den = px.density_heatmap(df.sample(50_000), x='x', y='y', nbinsx=200, nbinsy=200,
                            title='Density heatmap — sampled 50k')
fig_den.show()

## Model integration | Integração com modelos
Load a trained model from this project (example path) and visualize predictions. | Carregue um modelo treinado do projeto (caminho exemplo) e visualize previsões.

In [ ]:
# Example: load model artifact if exists (adapt to your project paths)
MODEL_PATH = os.getenv('MODEL_PATH', 'models/latest/model.pkl')
clf = None
try:
    import joblib
    if os.path.exists(MODEL_PATH):
        clf = joblib.load(MODEL_PATH)
        print('Loaded model from', MODEL_PATH)
    else:
        print('Model path not found, using synthetic predictions')
except Exception as e:
    print('Joblib not available or error loading model:', e)

X = df[['x','y']].head(2000).values
if clf is None:
    y_pred = (X[:,0] + X[:,1] > 0).astype(int)
else:
    try:
        y_pred = clf.predict(X)
    except Exception as e:
        print('Predict error, falling back to synthetic:', e)
        y_pred = (X[:,0] + X[:,1] > 0).astype(int)
pred_df = pd.DataFrame({'x': X[:,0], 'y': X[:,1], 'pred': y_pred})
px.scatter(pred_df, x='x', y='y', color='pred', title='Predictions overlay').show()

## Dash app | App Dash
Minimal app with controls and high-performance plots. | App mínimo com controles e gráficos de alta performance.

In [ ]:
# dash_app.py inside notebook cell (can be exported)
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'Advanced Viz — Dash'

app.layout = dbc.Container([
    html.H3('Advanced Visualization Dashboard — EN/PT'),
    dbc.Row([
        dbc.Col([
            dcc.Slider(id='n-sample', min=5_000, max=150_000, step=5_000, value=50_000),
            html.Small('Sample size | Tamanho da amostra')
        ], md=4),
        dbc.Col([
            dcc.Dropdown(id='render-mode', options=[
                {'label': 'WebGL', 'value': 'webgl'},
                {'label': 'SVG', 'value': 'auto'}
            ], value='webgl', clearable=False),
            html.Small('Render mode | Modo de renderização')
        ], md=4),
        dbc.Col([
            dcc.Checklist(id='show-density', options=[{'label':'Show density | Mostrar densidade','value':'yes'}], value=['yes'])
        ], md=4),
    ], className='my-2'),
    dbc.Row([
        dbc.Col(dcc.Graph(id='gl-scatter'), md=6),
        dbc.Col(dcc.Graph(id='density'), md=6),
    ])
], fluid=True)

@app.callback(
    Output('gl-scatter', 'figure'),
    Output('density', 'figure'),
    Input('n-sample', 'value'),
    Input('render-mode', 'value'),
    Input('show-density', 'value')
)
def update_figs(n_sample, render_mode, show_density):
    sample = df.sample(int(n_sample)) if n_sample < len(df) else df
    fig1 = px.scatter(sample, x='x', y='y', color='group', render_mode=render_mode,
                      title=f'Scatter — {len(sample):,} pts')
    if 'yes' in (show_density or []):
        fig2 = px.density_heatmap(sample, x='x', y='y', nbinsx=120, nbinsy=120,
                                 title='Density heatmap')
    else:
        fig2 = px.histogram2d(sample, x='x', y='y', nbinsx=120, nbinsy=120,
                             title='2D histogram')
    return fig1, fig2

# To run in notebook: app.run_server(debug=False, port=8050)
print('Dash app defined. Run with: app.run_server(port=8050)')

## Local run | Execução local
- python -m pip install -r requirements.txt
- python notebook_to_script.py 11_advanced_visualization_dash.ipynb  # optional export
- python -c "from app_dash import app; app.run_server(port=8050)"

## Cloud deploy (Render/Railway/Azure) | Deploy em nuvem (Render/Railway/Azure)
- Create Procfile: `web: gunicorn app_dash:app.server`
- requirements.txt includes: dash, dash-bootstrap-components, gunicorn, plotly
- Set PORT env var if required by platform
- For Azure Web App: configure Startup command to `gunicorn app_dash:app.server --bind=0.0.0.0:$PORT`